In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="iGSaRG2y43GaizViHQEx")
project = rf.workspace("machine-learning-class-eiri5").project("intersection-traffic-piimy")
version = project.version(8)
dataset = version.download("yolov8")
                

In [ ]:
# Импорты для работы с файлами, обработкой данных и обучения модели
import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# PyTorch и torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.models import resnet18

# Для визуализации эмбеддингов
from sklearn.manifold import TSNE


In [ ]:
# Пути к датасету
dataset_path = "/kaggle/working/Intersection-Traffic--8"
output_path = "/kaggle/working/intersection_classification"

# Создаем папки для классификационных данных
categories = ["bicycle", "bus", "car", "motorcycle"]
for category in categories:
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(output_path, split, category), exist_ok=True)

# Подготовка данных
def prepare_classification_data(dataset_path, output_path, test_size=0.2, val_size=0.1):
    image_paths = list(Path(dataset_path, "train/images").rglob("*.jpg"))
    labels_path = Path(dataset_path, "train/labels")

    # Формируем пары (изображение, метка)
    data = []
    for image_path in image_paths:
        label_path = labels_path / (image_path.stem + ".txt")
        if label_path.exists():
            with label_path.open("r") as f:
                lines = [line.strip().split() for line in f]
                if lines:
                    # Берем первую метку в качестве класса
                    class_idx = int(lines[0][0])
                    data.append((image_path, categories[class_idx]))

    # Разделяем на train, val, test
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=42)
    train_data, val_data = train_test_split(train_data, test_size=val_size, random_state=42)

    # Копирование данных
    def copy_data(split_data, split_name):
        for image_path, category in split_data:
            dest_path = Path(output_path, split_name, category, image_path.name)
            shutil.copy(image_path, dest_path)

    copy_data(train_data, "train")
    copy_data(val_data, "val")
    copy_data(test_data, "test")

# Запускаем подготовку данных
prepare_classification_data(dataset_path, output_path)


In [ ]:
# Гиперпараметры
batch_size = 32
learning_rate = 0.001
num_epochs = 30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Трансформации для данных
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Загрузка данных
train_dataset = datasets.ImageFolder(root=f"{output_path}/train", transform=transform)
val_dataset = datasets.ImageFolder(root=f"{output_path}/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Модель
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Инициализация модели
model = SimpleCNN(num_classes=len(categories)).to(device)

# Оптимизатор и функция потерь
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Функция обучения
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Прямой проход
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Обратный проход
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        # Валидация
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}, Accuracy: {100*correct/total:.2f}%")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100*val_correct/val_total:.2f}%")

# Запуск обучения
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)


In [ ]:
# Функция для извлечения эмбеддингов
def extract_embeddings(model, loader, device):
    model.eval()
    embeddings = []
    labels = []
    with torch.no_grad():
        for images, targets in loader:
            images = images.to(device)
            targets = targets.to(device)

            # Извлекаем эмбеддинги
            features = model.conv_layers(images)
            features = features.view(features.size(0), -1)
            embeddings.append(features.cpu().numpy())
            labels.append(targets.cpu().numpy())
    
    embeddings = np.concatenate(embeddings, axis=0)
    labels = np.concatenate(labels, axis=0)
    return embeddings, labels

# Извлечение эмбеддингов
train_embeddings, train_labels = extract_embeddings(model, train_loader, device)
val_embeddings, val_labels = extract_embeddings(model, val_loader, device)

# Визуализация с TSNE
def visualize_embeddings(embeddings, labels, title):
    tsne = TSNE(n_components=2, random_state=42)
    reduced_embeddings = tsne.fit_transform(embeddings)
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=labels, cmap='viridis', s=5)
    plt.colorbar(scatter, label="Class")
    plt.title(title)
    plt.xlabel("TSNE Component 1")
    plt.ylabel("TSNE Component 2")
    plt.show()

# Визуализация тренировочных эмбеддингов
visualize_embeddings(train_embeddings, train_labels, "Train Embeddings (TSNE)")

# Визуализация валидационных эмбеддингов
visualize_embeddings(val_embeddings, val_labels, "Validation Embeddings (TSNE)")


In [ ]:
# Загружаем предобученную модель
pretrained_model = resnet18(pretrained=True)

# Замораживаем веса backbone
for param in pretrained_model.parameters():
    param.requires_grad = False

# Добавляем кастомный классификатор
num_features = pretrained_model.fc.in_features
pretrained_model.fc = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, len(categories))
)
pretrained_model = pretrained_model.to(device)

# Оптимизатор и функция потерь
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.fc.parameters(), lr=learning_rate)

# Обучение модели Transfer Learning
train_model(pretrained_model, train_loader, val_loader, criterion, optimizer, num_epochs)


In [ ]:
import time

# Измерение времени обучения для CNN
start_time = time.time()
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)
cnn_time = time.time() - start_time

# Измерение времени обучения для Transfer Learning
start_time = time.time()
train_model(pretrained_model, train_loader, val_loader, criterion, optimizer, num_epochs)
transfer_time = time.time() - start_time

print(f"Simple CNN training time: {cnn_time:.2f} seconds")
print(f"Transfer Learning training time: {transfer_time:.2f} seconds")
